# Linear Adjustments

## Critical quantities

These 5 values need to be determined every time

* $n$: The number of observations (they are uncertain i.e. have covariance $\Sigma$ -- which is the point).
* $n_0$: The number of values (observations or parameters) that need to be fixed to determine 'the model'.
* $r$: Redundancy: $r=n-n_0$.
* $u$: Number of unknown parameters.
* $c$: Number of condition equations: $c=r+u$.

The first 3 are determined analytically, then $u$ and $c$ flow from them.

## Method of Indirect Observations
In this method we solve for the parameters of the model, so $u=n_0$. For instance for fitting a line, parameters are slope and intercept $m,b$, so $n_0=u=2$.

Once the $c=r+u$ condition equations are written, with all uncertain observations $\hat{l}_i$ expanded to solved observations plus residuals $l_i + v_i$, the linear system is formed/rearranged as $$v + B\Delta=f$$

Then the solution method is this series of matrix calculations:

* Weight matrix $W=\Sigma^{-1}$
* Normal equations: $N=B'WB$ and $t=B'Wf$
* Parameter solution: solve $N\Delta=t \Rightarrow \Delta = N^{-1}t$
* Residuals: $v=f-B\Delta$
* Covariance of parameters: $\Sigma_{\Delta\Delta} = N^{-1}$
* Covariance of residuals: $\Sigma_{vv} = \Sigma - BQ_{\Delta\Delta}B'$
* Adjusted observations: $\hat{l} = l + v$
* Covariance of $\hat{l}$: $\Sigma_{\hat{l}\hat{l}}=\Sigma-\Sigma{vv}$




In [3]:
import numpy as np

In [41]:
def print_indirect_observations(B, f, l=None, Q=None):
    c,u = B.shape
    r = c-u
    n0 = u      # always for indirect observations
    n  = n0 + r # always
    print('Problem shape:')
    print('  n:', n)
    print(' n0:', n0)
    print('  r:', r)
    print('  u:', u)
    print('  c:', c)
    
    print('Problem:')
    print('B:\n', B)
    print('f:', f.transpose())
    if Q is None: Q = np.eye(n)
    print('Q:\n', Q)
    
    print('\nNormal Equations:')
    W = np.linalg.inv(Q)
    print('Weight:\n',W)
    N = B.transpose() @ W @ B
    t = B.transpose() @ W @ f
    print('N:\n', N)
    print('t:', t.transpose())
    
    print('\nSolution')
    d = np.linalg.solve(N,t)
    print('Parameters:', d.transpose())
    Qdd = np.linalg.inv(N)
    print('Parameter covariance:\n',Qdd) 
    
    v = f - B@d
    print('Residuals:', v.transpose())
    Qvv = Q - B @ Qdd @ B.transpose()
    print('Residual covariance:\n', Qvv)
    
    if l is not None:
        lhat = l + v
        print('Original observations:', l.transpose())
        print('Adjusted observations:', lhat.transpose())
        Qll = Q - Qvv
        print('Covariance:\n',Qll)
    

Line fit example: points (1,1.1), (2,1.2), (3,1.5), (4,1.9) with $s_i=0.1$. 

* $n=4$: the four $y_i$
* $n_0=2$: line parameters $m,b$
* $r=2$
* $u=2$, we are solving for unknowns $\Delta=(m,b)$
* $c=4$, the line equations $\hat{y_i}=(y_i+v_i) = mx_i + b$

Condition equations reshape to $v+B\Delta=f$ as: $$v_i + -mx_i - b = -y_i$$

In [50]:
l = np.array([1.1, 1.2, 1.5, 1.9]).reshape( (4,1) )
B = np.array([ [-1,-1], [-2,-1], [-3,-1], [-4,-1] ])
f = -l
Q = np.eye(4) * 0.1*0.1
print_indirect_observations(B, f, -f, Q)

Problem shape:
  n: 4
 n0: 2
  r: 2
  u: 2
  c: 4
Problem:
B:
 [[-1 -1]
 [-2 -1]
 [-3 -1]
 [-4 -1]]
f: [[-1.1 -1.2 -1.5 -1.9]]
Q:
 [[0.01 0.   0.   0.  ]
 [0.   0.01 0.   0.  ]
 [0.   0.   0.01 0.  ]
 [0.   0.   0.   0.01]]

Normal Equations:
Weight:
 [[100.   0.   0.   0.]
 [  0. 100.   0.   0.]
 [  0.   0. 100.   0.]
 [  0.   0.   0. 100.]]
N:
 [[3000. 1000.]
 [1000.  400.]]
t: [[1560.  570.]]

Solution
Parameters: [[0.27 0.75]]
Parameter covariance:
 [[ 0.002 -0.005]
 [-0.005  0.015]]
Residuals: [[-0.08  0.09  0.06 -0.07]]
Residual covariance:
 [[ 0.003 -0.004 -0.001  0.002]
 [-0.004  0.007 -0.002 -0.001]
 [-0.001 -0.002  0.007 -0.004]
 [ 0.002 -0.001 -0.004  0.003]]
Original observations: [[1.1 1.2 1.5 1.9]]
Adjusted observations: [[1.02 1.29 1.56 1.83]]
Covariance:
 [[ 0.007  0.004  0.001 -0.002]
 [ 0.004  0.003  0.002  0.001]
 [ 0.001  0.002  0.003  0.004]
 [-0.002  0.001  0.004  0.007]]


## Method of Observations Only
In this method, the parameters are not solved for, so $u=0$ and thus $c=r$. All that is solved for is residuals (adjustments to the observations). The condition equations are shaped as $$Av=f$$

And then the solution method is:
* Equivalent covariance matrix: $\Sigma_e = A\Sigma A'$
* Equivalent weight matrix: $W_e = \Sigma_e^-1$
* Residuals: $v = \Sigma A'W_e f$
* Covariance of residuals: $\Sigma_{vv} = \Sigma A' W_e A \Sigma$
* Adjusted observations: $\hat{l} = l + v$
* Covariance of $\hat{l}$: $\Sigma_{\hat{l}\hat{l}}=\Sigma-\Sigma{vv}$


In [53]:
def print_observations_only(A,f,l=None,Q=None):
    c,n = A.shape
    u = 0     # always for observations only
    r = c-u   # always c=r+u
    n0 = n-r  # always r=n-n0
    print('Problem shape:')
    print('  n:', n)
    print(' n0:', n0)
    print('  r:', r)
    print('  u:', u)
    print('  c:', c)
    
    print('Problem:')
    print('A:\n', A)
    print('f:', f.transpose())
    if Q is None: Q = np.eye(n)
    print('Q:\n', Q)
    
    print('\nSolution:')
    Qe = A @ Q @ A.transpose()
    print('Equiv cov:\n', Qe)
    We = np.linalg.inv(Qe)
    print('Equiv wgt:\n', We)
    v = Q @ A.transpose() @ We @ f
    print('Residuals:', v.transpose())
    Qvv = Q @ A.transpose() @ We @ A @ Q
    print('Covariance:\n', Qvv)
    if l is not None:
        lhat = l + v
        print('Adjusted obs:', lhat.transpose())
        Qll = Q - Qvv
        print('Covariance:\n', Qll)

For the class Line fit example with observations only: points (1,1.1), (2,1.2), (3,1.5), (4,1.9) with $s_i=0.1$. 

* $n=4$: the four $y_i$
* $n_0=2$: 
* $r=2$
* $u=0$ always for Observations Only
* $c=r+u=2$, two equations to relate all the $y_i$ via slope conditions.

$$\frac{y_2-y_1}{2-1}=\frac{y_3-y_1}{3-1}$$
$$\frac{y_2-y_1}{2-1}=\frac{y_4-y_1}{4-1}$$ 

Those need to be rearranged to the form $Av=f$ 

$$2\hat{y}_2-2\hat{y}_1=\hat{y}_3-{y}_1\rightarrow -(y_1+v_1) + 2(y_2+v_2) - (y_3+v_3) = 0\rightarrow
-v_1 + 2v_2 - v_3 = y_1 - 2y_2 + y_3$$
$$3\hat{y}_2-3\hat{y}_1=\hat{y}_4-{y}_1\rightarrow -2(y_1+v_1) + 3(y_2+v_2) - (y_4+v_4) = 0\rightarrow
-2v_1 + 3v_2 - v_4 = 2y_1 - 3y_2 + y_4$$


In [54]:
A = np.array([[-1,2,-1,0],[-2,3,0,-1]])
f = np.array([[1.1-2*1.2+1.5], [2*1.1-3*1.2+1.9]])
print_observations_only(A,f,l,Q)

Problem shape:
  n: 4
 n0: 2
  r: 2
  u: 0
  c: 2
Problem:
A:
 [[-1  2 -1  0]
 [-2  3  0 -1]]
f: [[0.2 0.5]]
Q:
 [[0.01 0.   0.   0.  ]
 [0.   0.01 0.   0.  ]
 [0.   0.   0.01 0.  ]
 [0.   0.   0.   0.01]]

Solution:
Equiv cov:
 [[0.06 0.08]
 [0.08 0.14]]
Equiv wgt:
 [[ 70. -40.]
 [-40.  30.]]
Residuals: [[-0.08  0.09  0.06 -0.07]]
Covariance:
 [[ 0.003 -0.004 -0.001  0.002]
 [-0.004  0.007 -0.002 -0.001]
 [-0.001 -0.002  0.007 -0.004]
 [ 0.002 -0.001 -0.004  0.003]]
Adjusted obs: [[1.02 1.29 1.56 1.83]]
Covariance:
 [[ 0.007  0.004  0.001 -0.002]
 [ 0.004  0.003  0.002  0.001]
 [ 0.001  0.002  0.003  0.004]
 [-0.002  0.001  0.004  0.007]]


## General method
The General method combines both Indirect Observations and Observations only, for a full form of $$Av + B\Delta=f$$. It also requires initial estimates of the parameters, and iteration to convergence.

* Equivalent covariance matrix: $\Sigma_e = A\Sigma A'$
* Equivalent weight matrix: $W_e = \Sigma_e^-1$
* Normal equations: $N=B'W_eB$ and $t=B'W_e f$
* Parameter solution: $N\Delta=t$
* Residuals: $v=\Sigma A'W_e(f-B\Delta)$
* Adjusted observations: $\hat{l}=l+v$
* Covariance of Parameters: $\Sigma_{\Delta\Delta} = N^{-1}$
* Covariance of Residuals: $\Sigma_{vv} = \Sigma A'(W_e - W_eBN^{-1}B'W_e)A\Sigma$
* Covariance of Adjusted Observations: $\Sigma_{\hat{l}\hat{l}} = \Sigma - \Sigma_{vv}$



In [85]:
# For linear problems, apparently one iteration always suffices
# For nonlinear, we iterate til convergence

def general_iteration(A, B, f, l, Q, d, final=False):
    Qe = A @ Q @ A.transpose()
    We = np.linalg.inv(Qe)
    N = B.transpose() @ We @ B
    t = B.transpose() @ We @ f
    Qdd = np.linalg.inv(N)
    dd = Qdd @ t
    d += dd
    if final:
        print('Parameter corrections:', dd.transpose())
        print('Covariance:\n',Qdd)
        v = Q @ A.transpose() @ We @ (f - B@d)
        print('Residuals:', v.transpose())
        Qvv = Q @ A.transpose() @ (We-We@B@Qdd@B.transpose()@We) @ A @ Q
        print('Covariance:\n',Qvv)
        lhat = l + v
        print('Adjusted obs:', lhat.transpose())
        Qll = Q - Qvv
        print('Covariance:\n', Qll)
        
    return d 
    
    
    

## General method applied
Taking the line fit example from before, and letting the x's also be uncertain,
* $n=8$
* $n_0=6$: if we know all the $x_i$ and $m,b$ we can determine all the $y_i$
* $r=2$
* $u=2$: $m$ and $b$
* $c=4$: line equation applied to each point
Let's be naive and let $m_0=0$, $b_0=1.5$

In [83]:
m=0    # initialize to a horizontal line
b=1.5  # through kind of a middling height
d = np.array([m,b]).reshape((2,1))
l = np.array([1,2,3,4, 1.1,1.2,1.5,1.9]).reshape((8,1))
B=np.array([[-1,-1], [-2,-1], [-3,-1], [-4,-1]])

for i in range(2):
    A=np.array([ [-m,1,  0,0,0,0,0,0],
                 [0,0, -m,1, 0,0,0,0],
                 [0,0,0,0, -m,1, 0,0],
                 [0,0,0,0,0,0,  -m,1]])
    f=np.array([[-l[4,0] + m*l[0,0] + b],
                [-l[5,0] + m*l[1,0] + b],
                [-l[6,0] + m*l[2,0] + b],
                [-l[7,0] + m*l[3,0] + b] ])
    Q=np.eye(8)*0.1*0.1

    d = general_iteration(A,B,f,l,Q,d, i)
    m = d[0,0]
    b = d[1,0]


Parameter corrections: [[ 1.52468260e-16 -3.23995053e-16]]
Covariance:
 [[ 0.0021458 -0.0053645]
 [-0.0053645  0.0160935]]
Residuals: [[-0.23655513  0.87613011 -0.34728306  1.28623357 -0.40768012  1.50992637
  -0.44291173  1.64041383]]
Covariance:
 [[ 2.03840060e-04 -7.54963184e-04 -2.71786746e-04  1.00661758e-03
  -6.79466866e-05  2.51654395e-04  1.35893373e-04 -5.03308789e-04]
 [-7.54963184e-04  2.79615994e-03  1.00661758e-03 -3.72821325e-03
   2.51654395e-04 -9.32053313e-04 -5.03308789e-04  1.86410663e-03]
 [-2.71786746e-04  1.00661758e-03  4.75626806e-04 -1.76158076e-03
  -1.35893373e-04  5.03308789e-04 -6.79466866e-05  2.51654395e-04]
 [ 1.00661758e-03 -3.72821325e-03 -1.76158076e-03  6.52437319e-03
   5.03308789e-04 -1.86410663e-03  2.51654395e-04 -9.32053313e-04]
 [-6.79466866e-05  2.51654395e-04 -1.35893373e-04  5.03308789e-04
   4.75626806e-04 -1.76158076e-03 -2.71786746e-04  1.00661758e-03]
 [ 2.51654395e-04 -9.32053313e-04  5.03308789e-04 -1.86410663e-03
  -1.76158076e-03  6

In [86]:
d # the final solution for m,b, same as before

array([[0.27],
       [0.75]])

## HW4 

Use both methods to solve for the angles of a triangle given observed measurements of 70, 76, 35 deg, sigma 0.5.

### Indirect Observations

* $n=3$ angles measured
* $n_0=2$, any two true angles determines that the third is 180-
* $r=1$
* $u=2$, solve for angles 1 and 2
* $c=3$: each equation is simply $\hat{l}_i=\alpha_i$, except the third one is 180-

$$\begin{align}
v_1 + a_1 & = -l_1 \\
v_2 + a_2 & = -l_2 \\
v_3 - a_1 - a_2 & = -l_3 + 180
\end{align}$$

In [55]:
l = np.array([70,76,35]).reshape( (3,1) )
B=np.array([[1,0],[0,1],[-1,-1]])
f = -l
f[2,0] += 180
Q = np.eye(3)*0.5*0.5
print_indirect_observations(B,f,l,Q)

Problem shape:
  n: 3
 n0: 2
  r: 1
  u: 2
  c: 3
Problem:
B:
 [[ 1  0]
 [ 0  1]
 [-1 -1]]
f: [[-70 -76 145]]
Q:
 [[0.25 0.   0.  ]
 [0.   0.25 0.  ]
 [0.   0.   0.25]]

Normal Equations:
Weight:
 [[4. 0. 0.]
 [0. 4. 0.]
 [0. 0. 4.]]
N:
 [[8. 4.]
 [4. 8.]]
t: [[-860. -884.]]

Solution
Parameters: [[-69.66666667 -75.66666667]]
Parameter covariance:
 [[ 0.16666667 -0.08333333]
 [-0.08333333  0.16666667]]
Residuals: [[-0.33333333 -0.33333333 -0.33333333]]
Residual covariance:
 [[0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]]
Original observations: [[70 76 35]]
Adjusted observations: [[69.66666667 75.66666667 34.66666667]]
Covariance:
 [[ 0.16666667 -0.08333333 -0.08333333]
 [-0.08333333  0.16666667 -0.08333333]
 [-0.08333333 -0.08333333  0.16666667]]


### Observations only

As before, $n=3, n_0=2, r=1$, but with $u=0$, $c=r+u=1$. The one equation is that the sum of the angles is 180.

$$\hat{l}_1 + \hat{l}_2 + \hat{l}_3 = 180$$
$$v_1 + v_2 + v_3 = 180 - l_1 - l_2 - l_3$$



In [56]:
B = np.array([[1,1,1]])
f = np.array([[180 - np.sum(l)]])
print_observations_only(B,f,l,Q)

Problem shape:
  n: 3
 n0: 2
  r: 1
  u: 0
  c: 1
Problem:
A:
 [[1 1 1]]
f: [[-1]]
Q:
 [[0.25 0.   0.  ]
 [0.   0.25 0.  ]
 [0.   0.   0.25]]

Solution:
Equiv cov:
 [[0.75]]
Equiv wgt:
 [[1.33333333]]
Residuals: [[-0.33333333 -0.33333333 -0.33333333]]
Covariance:
 [[0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]]
Adjusted obs: [[69.66666667 75.66666667 34.66666667]]
Covariance:
 [[ 0.16666667 -0.08333333 -0.08333333]
 [-0.08333333  0.16666667 -0.08333333]
 [-0.08333333 -0.08333333  0.16666667]]
